# Exploratory Data Analysis

In this notebook I'll conduct an exploration of the "Maven Fuzzy Factory" dataset. This includes profiling, cleaning, and visualizing the data. Although the data is synthetic (therefore not real business data) it can be useful for demonstrating my skills as a data analyst. 

TO DO: ERD of the database in the Reading the Data Dictionary section, please.

## About the data

"Maven Fuzzy Factory" is the name of a fictitious E-Commerce website selling stuffed animals. The [original dataset](https://mavenanalytics.io/data-playground/toy-store-e-commerce-database) was created by Maven Analytics.

## Relevant business questions to answer

For an e-commerce website some relevant metrics are:
* Most popular product
* Least popular product
* Conversion rate
* Average order value

Additional questions can be formulated after skimming over the data.

## Importing required libraries

In [4]:
import pandas as pd

## Reading the data dictionary

The dataset comes with a data dictionary detailing the relationships between the different tables.

In [30]:
data_dict = pd.read_csv('raw/maven_fuzzy_factory_data_dictionary.csv')
data_dict.info()
data_dict.head()

<class 'pandas.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Table        36 non-null     str  
 1   Field        36 non-null     str  
 2   Description  36 non-null     str  
dtypes: str(3)
memory usage: 996.0 bytes


,Table,Field,Description
0,orders,order_id,Unique identifier for each order (PK)
1,orders,created_at,Timestamp when the order was placed
2,orders,website_session_id,Unique identifier for the website session (FK)
3,orders,user_id,Unique identifier for the user (FK)
4,orders,primary_product_id,Unique identifier for the primary product in t...


Seems like the data dictionary loaded correctly. How many tables does the dictionary describe?

In [32]:
count = 0
for table in list(data_dict['Table'].unique()):
    print('\''+table+'\'')
    count += 1
print(f'tables describe: {count}')

'orders'
'order_items'
'order_item_refunds'
'products'
'website_sessions'
'website_pageviews'
tables describe: 6


The data dictionary packaged with the database seems to cover all (6) available files, let's explore them one by one.

What are the characteristics of the `orders` table?

In [40]:
descriptions = {}
descriptions['orders'] = data_dict[data_dict['Table'] == 'orders']
descriptions['orders']

,Table,Field,Description
0,orders,order_id,Unique identifier for each order (PK)
1,orders,created_at,Timestamp when the order was placed
2,orders,website_session_id,Unique identifier for the website session (FK)
3,orders,user_id,Unique identifier for the user (FK)
4,orders,primary_product_id,Unique identifier for the primary product in t...
5,orders,items_purchased,Number of items in the order
6,orders,price_usd,Total price for the items in the order
7,orders,cogs_usd,Cost of goods sold for the items in the order


What are the characteristics of the `order_items` table?

In [41]:
descriptions['order_items'] = data_dict[data_dict['Table'] == 'order_items']
descriptions['order_items']

,Table,Field,Description
8,order_items,order_item_id,Unique identifier for each order item (PK)
9,order_items,created_at,Timestamp when the order was placed
10,order_items,order_id,Unique identifier for the order the item belon...
11,order_items,product_id,Unique identifier for the product (FK)
12,order_items,is_primary_item,Binary flag with a value of 1 if it's the prim...
13,order_items,price_usd,Price of the product
14,order_items,cogs_usd,Cost of goods sold of the product


What are the characteristics of the `order_item_refunds` table?

In [42]:
descriptions['order_item_refunds'] = data_dict[data_dict['Table'] == 'order_item_refunds']
descriptions['order_item_refunds']

,Table,Field,Description
15,order_item_refunds,order_item_refund_id,Unique identifier for each refund (PK)
16,order_item_refunds,created_at,Timestamp when the refund was issued
17,order_item_refunds,order_item_id,Unique identifier for the order item that was ...
18,order_item_refunds,order_id,Unique identifier for the order the refunded i...
19,order_item_refunds,refund_amount_usd,Refund amount


What are the characteristic of the `products` table?

In [43]:
descriptions['products'] = data_dict[data_dict['Table'] == 'products']
descriptions['products'] 

,Table,Field,Description
20,products,product_id,Unique identifier for the product (PK)
21,products,created_at,Timestamp for when the product was launched
22,products,product_name,Name of the product


What are the characteristics of the `website_sessions` table?

In [45]:
descriptions['website_sessions'] = data_dict[data_dict['Table'] == 'website_sessions']
descriptions['website_sessions']

,Table,Field,Description
23,website_sessions,website_session_id,Unique identifier for the website session (PK)
24,website_sessions,created_at,Timestamp when the session started
25,website_sessions,user_id,Unique identifier for the user (FK)
26,website_sessions,is_repeat_session,Binary flag with a value of 1 if the user has ...
27,website_sessions,utm_source,UTM source parameter (traffic origin)
28,website_sessions,utm_campaign,UTM campaign parameter (marketing campaign name)
29,website_sessions,utm_content,UTM content parameter (ad/content variant)
30,website_sessions,device_type,Device category (mobile or desktop)
31,website_sessions,http_referer,URL for the UTM source


What are the characteristics of the `website_pageviews` table?

In [46]:
descriptions['website_pageviews'] = data_dict[data_dict['Table'] == 'website_pageviews']
descriptions['website_pageviews']

,Table,Field,Description
32,website_pageviews,website_pageview_id,Unique identifier for each website pageview (PK)
33,website_pageviews,created_at,Timestamp for the pageview
34,website_pageviews,website_session_id,Unique identifier for the website session the ...
35,website_pageviews,pageview_url,URL path for the pageview


### Results

I have loaded the descriptions of the entire dataset into a python dictionary object to work with as I profile the actual data later in this document:
* The `csv` data dictionary provided by the authors loaded correctly and contains the context of the dataset.
* I've found the correct data types of all columns in the data.
* I now understand the relationship between the tables and can create an ERD (entity relationship diagram).

## Extracting the rest of the tables

In [50]:
orders = pd.read_csv('raw/orders.csv')
order_items = pd.read_csv('raw/order_items.csv')
order_item_refunds = pd.read_csv('raw/order_item_refunds.csv')
products = pd.read_csv('raw/products.csv')
website_pageviews = pd.read_csv('raw/website_pageviews.csv')
website_sessions = pd.read_csv('raw/website_sessions.csv')

## Data profiling

In this section I'll proceed to check the data for specific errors, flaws, and/or possible contradictions. Correct datatypes will be asigned to all columns according to the context in the data dictionary. 

### Table: `orders`

In [52]:
descriptions['orders']

,Table,Field,Description
0,orders,order_id,Unique identifier for each order (PK)
1,orders,created_at,Timestamp when the order was placed
2,orders,website_session_id,Unique identifier for the website session (FK)
3,orders,user_id,Unique identifier for the user (FK)
4,orders,primary_product_id,Unique identifier for the primary product in t...
5,orders,items_purchased,Number of items in the order
6,orders,price_usd,Total price for the items in the order
7,orders,cogs_usd,Cost of goods sold for the items in the order


In [51]:
orders.info()
orders.head()

<class 'pandas.DataFrame'>
RangeIndex: 32313 entries, 0 to 32312
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   order_id            32313 non-null  int64  
 1   created_at          32313 non-null  str    
 2   website_session_id  32313 non-null  int64  
 3   user_id             32313 non-null  int64  
 4   primary_product_id  32313 non-null  int64  
 5   items_purchased     32313 non-null  int64  
 6   price_usd           32313 non-null  float64
 7   cogs_usd            32313 non-null  float64
dtypes: float64(2), int64(5), str(1)
memory usage: 2.0 MB


,order_id,created_at,website_session_id,user_id,primary_product_id,items_purchased,price_usd,cogs_usd
0,1,2012-03-19 10:42:46,20,20,1,1,49.99,19.49
1,2,2012-03-19 19:27:37,104,104,1,1,49.99,19.49
2,3,2012-03-20 06:44:45,147,147,1,1,49.99,19.49
3,4,2012-03-20 09:41:45,160,160,1,1,49.99,19.49
4,5,2012-03-20 11:28:15,177,177,1,1,49.99,19.49


#### Profiling results, `orders`

* The data loaded correctly and conforms to the structure mentioned in the data dictionary.
* The data contains no missing or null values, explicit or implicit.
* TO DO adjust data types, check for duplicates, check for outliers

## Univariate Analysis

## Bivariate Analysis